In [1]:
from numpy.random import seed
seed(1)
import pandas as pd
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# loading data into a dataframe

file_path = '../Resources/exoplanet_data.csv'

df = pd.read_csv(file_path)
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

koi_disposition  koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  \
0       CONFIRMED              0              0              0              0   
1  FALSE POSITIVE              0              1              0              0   
2  FALSE POSITIVE              0              1              0              0   
3       CONFIRMED              0              0              0              0   
4       CONFIRMED              0              0              0              0   

   koi_period  koi_period_err1  koi_period_err2  koi_time0bk  \
0   54.418383     2.479000e-04    -2.479000e-04   162.513840   
1   19.899140     1.490000e-05    -1.490000e-05   175.850252   
2    1.736952     2.630000e-07    -2.630000e-07   170.307565   
3    2.525592     3.760000e-06    -3.760000e-06   171.595550   
4    4.134435     1.050000e-05    -1.050000e-05   172.979370   

   koi_time0bk_err1  ...  koi_steff_err2  koi_slogg  koi_slogg_err1  \
0          0.003520  ...             -81      4.467           0.064   
1          0.000581  ...            -176      4.544           0.044   
2          0.000115  ...            -174      4.564           0.053   
3          0.001130  ...            -211      4.438           0.070   
4          0.001900  ...            -232      4.486           0.054   

   koi_slogg_err2  koi_srad  koi_srad_err1  koi_srad_err2         ra  \
0          -0.096     0.927          0.105         -0.061  291.93423   
1          -0.176     0.868          0.233         -0.078  297.00482   
2          -0.168     0.791          0.201         -0.067  285.53461   
3          -0.210     1.046          0.334         -0.133  288.75488   
4          -0.229     0.972          0.315         -0.105  296.28613   

         dec  koi_kepmag  
0  48.141651      15.347  
1  48.134129      15.436  
2  48.285210      15.597  
3  48.226200      15.509  
4  48.224670      15.714  

[5 rows x 41 columns]

In [3]:
# Setting features and target
target = df['koi_disposition']
selected_features = df.drop(['koi_disposition'], axis=1)

# Selected Feature Names
feature_names = selected_features.columns.tolist()

# looking at the shape of the variables
print(target.shape, selected_features.shape)

(6991,) (6991, 40)


In [4]:
# Splitting the data into a training set and testing set and scaling

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Encoding the target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Converting the encoded target variable into a catagorical form
y_train_catagorical = to_categorical(encoded_y_train)
y_test_catagorical = to_categorical(encoded_y_test)

In [5]:
# Creating the deep model

deep_model = Sequential()
deep_model.add(Dense(units=10, activation='relu', input_dim=40))
deep_model.add(Dense(units=10, activation='relu'))
deep_model.add(Dense(units=3, activation='softmax'))

In [6]:
# Compiling the model

deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model.fit(X_train_scaled, y_train_catagorical, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
164/164 - 0s - loss: 0.9578 - accuracy: 0.4921
Epoch 2/100
164/164 - 0s - loss: 0.5860 - accuracy: 0.7461
Epoch 3/100
164/164 - 0s - loss: 0.4308 - accuracy: 0.7919
Epoch 4/100
164/164 - 0s - loss: 0.3924 - accuracy: 0.8068
Epoch 5/100
164/164 - 0s - loss: 0.3800 - accuracy: 0.8085
Epoch 6/100
164/164 - 0s - loss: 0.3731 - accuracy: 0.8062
Epoch 7/100
164/164 - 0s - loss: 0.3649 - accuracy: 0.8201
Epoch 8/100
164/164 - 0s - loss: 0.3603 - accuracy: 0.8161
Epoch 9/100
164/164 - 0s - loss: 0.3549 - accuracy: 0.8236
Epoch 10/100
164/164 - 0s - loss: 0.3502 - accuracy: 0.8245
Epoch 11/100
164/164 - 0s - loss: 0.3450 - accuracy: 0.8259
Epoch 12/100
164/164 - 0s - loss: 0.3427 - accuracy: 0.8310
Epoch 13/100
164/164 - 0s - loss: 0.3370 - accuracy: 0.8346
Epoch 14/100
164/164 - 0s - loss: 0.3349 - accuracy: 0.8362
Epoch 15/100
164/164 - 0s - loss: 0.3327 - accuracy: 0.8352
Epoch 16/100
164/164 - 0s - loss: 0.3319 - accuracy: 0.8379
Epoch 17/100
164/164 - 0s - loss: 0.3245 - accura

In [7]:
# scoring the model

deep_model_loss, deep_model_accuracy = deep_model.evaluate(X_test_scaled, y_test_catagorical, verbose=2)
print(f"Normal Neural Network - Loss: {deep_model_loss}, Accuracy: {deep_model_accuracy}")

55/55 - 0s - loss: 0.2913 - accuracy: 0.8781
Normal Neural Network - Loss: 0.2912806570529938, Accuracy: 0.8781464695930481


In [8]:
# Creating a model with more neural layers

deep_model2 = Sequential()
deep_model2.add(Dense(units=10, activation='relu', input_dim=40))
deep_model2.add(Dense(units=10, activation='relu'))
deep_model2.add(Dense(units=10, activation='relu'))
deep_model2.add(Dense(units=10, activation='relu'))
deep_model2.add(Dense(units=10, activation='relu'))
deep_model2.add(Dense(units=3, activation='softmax'))


In [9]:
# Compiling the model with more layers

deep_model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model2.fit(X_train_scaled, y_train_catagorical, epochs=100, shuffle=True, verbose=2)

Epoch 1/100
164/164 - 0s - loss: 0.8425 - accuracy: 0.5745
Epoch 2/100
164/164 - 0s - loss: 0.4477 - accuracy: 0.7545
Epoch 3/100
164/164 - 0s - loss: 0.3896 - accuracy: 0.7877
Epoch 4/100
164/164 - 0s - loss: 0.3801 - accuracy: 0.7992
Epoch 5/100
164/164 - 0s - loss: 0.3718 - accuracy: 0.8072
Epoch 6/100
164/164 - 0s - loss: 0.3709 - accuracy: 0.8085
Epoch 7/100
164/164 - 0s - loss: 0.3628 - accuracy: 0.8182
Epoch 8/100
164/164 - 0s - loss: 0.3612 - accuracy: 0.8127
Epoch 9/100
164/164 - 0s - loss: 0.3581 - accuracy: 0.8129
Epoch 10/100
164/164 - 0s - loss: 0.3548 - accuracy: 0.8213
Epoch 11/100
164/164 - 0s - loss: 0.3512 - accuracy: 0.8285
Epoch 12/100
164/164 - 0s - loss: 0.3500 - accuracy: 0.8264
Epoch 13/100
164/164 - 0s - loss: 0.3514 - accuracy: 0.8236
Epoch 14/100
164/164 - 0s - loss: 0.3470 - accuracy: 0.8280
Epoch 15/100
164/164 - 0s - loss: 0.3461 - accuracy: 0.8262
Epoch 16/100
164/164 - 0s - loss: 0.3436 - accuracy: 0.8322
Epoch 17/100
164/164 - 0s - loss: 0.3444 - accura

In [10]:
# scoring the model with more layers

deep_model2_loss, deep_model2_accuracy = deep_model2.evaluate(X_test_scaled, y_test_catagorical, verbose=2)
print(f"Normal Neural Network - Loss: {deep_model2_loss}, Accuracy: {deep_model2_accuracy}")

55/55 - 0s - loss: 0.3000 - accuracy: 0.8707
Normal Neural Network - Loss: 0.29997047781944275, Accuracy: 0.8707093596458435


In [11]:
# Finding most important features

feat_model = RandomForestClassifier()
feat_model.fit(X_train_scaled, y_train_catagorical)
feats = feat_model.feature_importances_
feat_list = sorted(zip(feats,feature_names), reverse=True)
feat_list

[(0.10609241249936005, 'koi_fpflag_nt'),
 (0.10490828383393712, 'koi_fpflag_co'),
 (0.07239708426099763, 'koi_fpflag_ss'),
 (0.053618395316294185, 'koi_prad'),
 (0.052256945280473734, 'koi_model_snr'),
 (0.040349159298171296, 'koi_duration_err2'),
 (0.03522721436727802, 'koi_fpflag_ec'),
 (0.034591183465736794, 'koi_duration_err1'),
 (0.03265592411164437, 'koi_steff_err1'),
 (0.03154042362046396, 'koi_prad_err2'),
 (0.02425896131747935, 'koi_prad_err1'),
 (0.02259951924316135, 'koi_steff_err2'),
 (0.022265663734593964, 'koi_duration'),
 (0.022214998514511888, 'koi_depth'),
 (0.0213132689302937, 'koi_period'),
 (0.020792771869247585, 'koi_insol_err1'),
 (0.020656788233389886, 'koi_time0bk_err1'),
 (0.019599388860085623, 'koi_impact'),
 (0.0192262905932186, 'koi_time0bk_err2'),
 (0.0176505973771433, 'koi_period_err2'),
 (0.017628791439029377, 'koi_period_err1'),
 (0.016183682397578572, 'koi_teq'),
 (0.013929580749234182, 'koi_time0bk'),
 (0.013882238460555052, 'koi_depth_err2'),
 (0.0137

In [12]:
# creating a selected features variable with only the top 15 most important variables

short_feat_list = feat_list[:15]
short_feat_list = list(zip(*short_feat_list))
short_feat_list = list(short_feat_list[1])
tuned_X = df[short_feat_list]
tuned_X.shape

(6991, 15)

In [14]:
# creating a deep model with the most important variables

# Spliting the tuned data
tuned_X_train, tuned_X_test, y_train, y_test = train_test_split(tuned_X, target, random_state=42)

# Scaling the tuned data
tuned_X_scaler = MinMaxScaler().fit(tuned_X_train)

tuned_X_train_scaled = tuned_X_scaler.transform(tuned_X_train)
tuned_X_test_scaled = tuned_X_scaler.transform(tuned_X_test)

# Retraining the tuned data
tuned_deep_model = Sequential()
tuned_deep_model.add(Dense(units=10, activation='relu', input_dim=15))
tuned_deep_model.add(Dense(units=10, activation='relu'))
tuned_deep_model.add(Dense(units=10, activation='relu'))
tuned_deep_model.add(Dense(units=10, activation='relu'))
tuned_deep_model.add(Dense(units=10, activation='relu'))
tuned_deep_model.add(Dense(units=3, activation='softmax'))

# Compiling the tuned data
tuned_deep_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
tuned_deep_model.fit(tuned_X_train_scaled, y_train_catagorical, epochs=100, shuffle=True, verbose=2)



Epoch 1/100
164/164 - 0s - loss: 0.9598 - accuracy: 0.5977
Epoch 2/100
164/164 - 0s - loss: 0.4890 - accuracy: 0.7452
Epoch 3/100
164/164 - 0s - loss: 0.4033 - accuracy: 0.7463
Epoch 4/100
164/164 - 0s - loss: 0.3898 - accuracy: 0.7776
Epoch 5/100
164/164 - 0s - loss: 0.3843 - accuracy: 0.7858
Epoch 6/100
164/164 - 0s - loss: 0.3791 - accuracy: 0.7860
Epoch 7/100
164/164 - 0s - loss: 0.3749 - accuracy: 0.7934
Epoch 8/100
164/164 - 0s - loss: 0.3712 - accuracy: 0.8049
Epoch 9/100
164/164 - 0s - loss: 0.3668 - accuracy: 0.8039
Epoch 10/100
164/164 - 0s - loss: 0.3613 - accuracy: 0.8190
Epoch 11/100
164/164 - 0s - loss: 0.3578 - accuracy: 0.8200
Epoch 12/100
164/164 - 0s - loss: 0.3511 - accuracy: 0.8228
Epoch 13/100
164/164 - 0s - loss: 0.3483 - accuracy: 0.8270
Epoch 14/100
164/164 - 0s - loss: 0.3459 - accuracy: 0.8291
Epoch 15/100
164/164 - 0s - loss: 0.3436 - accuracy: 0.8327
Epoch 16/100
164/164 - 0s - loss: 0.3393 - accuracy: 0.8354
Epoch 17/100
164/164 - 0s - loss: 0.3371 - accura

In [15]:
# Scoring the tuned data
tuned_deep_model_loss, tuned_deep_model_accuracy = tuned_deep_model.evaluate(tuned_X_test_scaled, y_test_catagorical, verbose=2)
print(f"Normal Neural Network - Loss: {tuned_deep_model_loss}, Accuracy: {tuned_deep_model_accuracy}")

55/55 - 0s - loss: 0.3009 - accuracy: 0.8741
Normal Neural Network - Loss: 0.30088040232658386, Accuracy: 0.8741418719291687
